In [2]:
import dlib,cv2
import numpy as np

In [3]:
model_path = "shape_predictor_68_face_landmarks.dat"

In [4]:
frontalFaceDetector = dlib.get_frontal_face_detector()

In [5]:
faceLandMarkDetector = dlib.shape_predictor(model_path)

In [6]:
imgGlass = cv2.imread('glasses-2.png',-1)

glass_img_height,glass_img_width = imgGlass.shape[0:2]
orig_mask = imgGlass[:,:,3]
orig_mask_inv = cv2.bitwise_not(orig_mask)
imgGlass = imgGlass[:,:,0:3]
# cv2.imshow("glass",orig_mask)
# cv2.imshow("glass_inv",orig_mask_inv)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
# camera = cv2.VideoCapture(0)
# frontalFaceDetector = dlib.get_frontal_face_detector()
# while True:
#     return_value, image = camera.read()
#     allFaces = frontalFaceDetector(image,1)
#     for k in range(0, len(allFaces)):
#         image_show = cv2.rectangle(image,(int(allFaces[k].left()),int(allFaces[k].top())),(int(allFaces[k].right()),int(allFaces[k].bottom())),(0,255,0),3)
#         cv2.imshow("image",image_show)
#         if(cv2.waitKey(1)==13):
#             break
# camera.release()
# cv2.destroyAllWindows()

In [7]:
def drawPoints(image,faceLandmarks,startpoint,endpoint,isClosed=False):
    points = []
    for i in range(startpoint,endpoint+1):
        point = [faceLandmarks.part(i).x,faceLandmarks.part(i).y]
        points.append(point)
    points = np.array(points,dtype="int32")
    cv2.polylines(image,[points],isClosed,(255,204,0),thickness=2,lineType=cv2.LINE_8)
    
    
def facePoints(image, faceLandmarks):
    assert(faceLandmarks.num_parts == 68)
    drawPoints(image, faceLandmarks, 0, 16)           # Jaw line
    drawPoints(image, faceLandmarks, 17, 21)          # Left eyebrow
    drawPoints(image, faceLandmarks, 22, 26)          # Right eyebrow
    drawPoints(image, faceLandmarks, 27, 30)          # Nose bridge
    drawPoints(image, faceLandmarks, 30, 35, True)    # Lower nose
    drawPoints(image, faceLandmarks, 36, 41, True)    # Left eye
    drawPoints(image, faceLandmarks, 42, 47, True)    # Right Eye
    drawPoints(image, faceLandmarks, 48, 59, True)    # Outer lip
    drawPoints(image, faceLandmarks, 60, 67, True)
    
def getGlasses(frame,faceLandmarks):
    glassWidth = abs(faceLandmarks.part(16).x - faceLandmarks.part(1).x)
    glassHeight = int((glass_img_height / glass_img_width) * glassWidth)
    glass = cv2.resize(imgGlass,(glassWidth,glassHeight),interpolation=cv2.INTER_AREA)
    mask = cv2.resize(orig_mask,(glassWidth,glassHeight),interpolation=cv2.INTER_AREA)
    mask_inv = cv2.resize(orig_mask_inv,(glassWidth,glassHeight),interpolation=cv2.INTER_AREA)
    
    y1 = int(faceLandmarks.part(24).y)
    y2 = int(y1 + glassHeight)
    x1 = int(faceLandmarks.part(27).x - (glassWidth/2))
    x2 = int(x1 + glassWidth)
    points = [[]]
    cv2.polylines(image,)
#     cv2.rectangle(frame,(x1,y1),(x2,y2),(255,0,123),2,2)
    
#     roi1 = frame[y1:y2, x1:x2]
#     roi_bg = cv2.bitwise_and(roi1,roi1,mask = mask_inv)
#     roi_fg = cv2.bitwise_and(glass,glass,mask = mask)
#     frame[y1:y2, x1:x2] = cv2.add(roi_fg,roi_bg)
    
    return frame

    
    
    

In [8]:
camera = cv2.VideoCapture(0)
frontalFaceDetector = dlib.get_frontal_face_detector()
while True:
    ret,image = camera.read()
    allFaces = frontalFaceDetector(image,1)
    allFacesLandmark = []
    for k in range(0,len(allFaces)):
        faceRectangleDlib = dlib.rectangle(int(allFaces[k].left()),int(allFaces[k].top()),int(allFaces[k].right()),int(allFaces[k].bottom()))
        detectedLandMarks = faceLandMarkDetector(image,faceRectangleDlib)
#         facePoints(image,detectedLandMarks)
        getGlasses(image,detectedLandMarks)
    cv2.imshow("camera", image)
    if(cv2.waitKey(1)==13):
        break
camera.release()
cv2.destroyAllWindows()